In [3]:
# -*- coding: utf-8 -*-

# License: BSD
# Author: Ghassen Hamrouni

from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np
import scipy.misc

plt.ion()   # interactive mode

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Training dataset
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST(root='.', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])), batch_size=64, shuffle=True, num_workers=4)
# Test dataset
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST(root='.', train=False, transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])), batch_size=64, shuffle=True, num_workers=4)

In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

        # Spatial transformer localization-network
        self.localization = nn.Sequential(
            nn.Conv2d(1, 8, kernel_size=7),
            nn.MaxPool2d(2, stride=2),
            nn.ReLU(True),
            nn.Conv2d(8, 10, kernel_size=5),
            nn.MaxPool2d(2, stride=2),
            nn.ReLU(True)
        )

        # Regressor for the 3 * 2 affine matrix
        self.fc_loc = nn.Sequential(
            nn.Linear(10 * 3 * 3, 32),
            nn.ReLU(True),
            nn.Linear(32, 3 * 2)
        )

        # Initialize the weights/bias with identity transformation
        self.fc_loc[2].weight.data.zero_()
        self.fc_loc[2].bias.data.copy_(torch.tensor([1, 0, 0, 0, 1, 0], dtype=torch.float))

    # Spatial transformer network forward function
    def stn(self, x):
        xs = self.localization(x)
        xs = xs.view(-1, 10 * 3 * 3)
        theta = self.fc_loc(xs)
        theta = theta.view(-1, 2, 3)

        grid = F.affine_grid(theta, x.size())
        x = F.grid_sample(x, grid)

        return x

    def forward(self, x):
        # transform the input
        x = self.stn(x)

        # Perform the usual forward pass
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)


model = Net().to(device)

In [5]:
optimizer = optim.SGD(model.parameters(), lr=0.01)


def train(epoch):
    model.train()
    total_loss = []
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 500 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
                total_loss.append(loss.item)
            
#
# A simple test procedure to measure STN the performances on MNIST.
#


def test():
    with torch.no_grad():
        model.eval()
        test_loss = 0
        correct = 0
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)

            # sum up batch loss
            test_loss += F.nll_loss(output, target, size_average=False).item()
            # get the index of the max log-probability
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()

        test_loss /= len(test_loader.dataset)
        print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'
              .format(test_loss, correct, len(test_loader.dataset),
                      100. * correct / len(test_loader.dataset)))

######################################################################
# Visualizing the STN results
# ---------------------------
#
# Now, we will inspect the results of our learned visual attention
# mechanism.
#
# We define a small helper function in order to visualize the
# transformations while training.


def convert_image_np(inp):
    """Convert a Tensor to numpy image."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    return inp

# We want to visualize the output of the spatial transformers layer
# after the training, we visualize a batch of input images and
# the corresponding transformed batch using STN.


def visualize_stn():
    with torch.no_grad():
        # Get a batch of training data
        data = next(iter(test_loader))[0].to(device)

        input_tensor = data.cpu()
        transformed_input_tensor = model.stn(data).cpu()

        in_grid = convert_image_np(
            torchvision.utils.make_grid(input_tensor))

        out_grid = convert_image_np(
            torchvision.utils.make_grid(transformed_input_tensor))

        scipy.misc.imsave('in_grid.png', in_grid)
        scipy.misc.imsave('out_grid.png', out_grid)
        torchvision.utils.save_image(input_tensor, "in_grid_img.png", nrow=8, padding=2, normalize=False, range=None, scale_each=False,
                                     pad_value=0)

        torchvision.utils.save_image(transformed_input_tensor, "out_grid_img.png", nrow=8, padding=2, normalize=False, range=None,
                                     scale_each=False,
                                     pad_value=0)


        # # Plot the results side-by-side
        # f, axarr = plt.subplots(1, 2)
        # axarr[0].imshow(in_grid)
        # axarr[0].set_title('Dataset Images')
        #
        # axarr[1].imshow(out_grid)
        # axarr[1].set_title('Transformed Images')


for epoch in range(1, 20 + 1):
    train(epoch)
    test()


# Visualize the STN transformation on some input batch
visualize_stn()

plt.ioff()
plt.show()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.300849
Train Epoch: 1 [32000/60000 (53%)]	Loss: 1.030376

Test set: Average loss: 0.2591, Accuracy: 9227/10000 (92%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.477124
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.264399

Test set: Average loss: 0.3328, Accuracy: 8893/10000 (89%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.715932
Train Epoch: 3 [32000/60000 (53%)]	Loss: 0.428384

Test set: Average loss: 0.1011, Accuracy: 9714/10000 (97%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 0.197246
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.225891

Test set: Average loss: 0.1043, Accuracy: 9681/10000 (97%)

Train Epoch: 5 [0/60000 (0%)]	Loss: 0.338069
Train Epoch: 5 [32000/60000 (53%)]	Loss: 0.101722

Test set: Average loss: 0.0797, Accuracy: 9759/10000 (98%)

Train Epoch: 6 [0/60000 (0%)]	Loss: 0.300147
Train Epoch: 6 [32000/60000 (53%)]	Loss: 0.367683

Test set: Average loss: 0.0869, Accuracy: 9740/10000 (97%)

Train Epoch: 7 [0/60000 (0%)]	Loss: 0.084209
Train Epoch: 7 [320

Process Process-100:
Process Process-99:
Process Process-97:
Process Process-98:
Traceback (most recent call last):
  File "/home/imatge_upc_ai/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/imatge_upc_ai/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/imatge_upc_ai/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 57, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/home/imatge_upc_ai/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 138, in default_collate
    return [default_collate(samples) for samples in transposed]
  File "/home/imatge_upc_ai/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 138, in <listcomp>
    return [default_collate(samples) for samples in transposed]
  File "/home/imatge_upc_ai/anaconda3/lib/python3.6/site

Traceback (most recent call last):
  File "/home/imatge_upc_ai/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2963, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-5-e8d018404dec>", line 102, in <module>
    train(epoch)
  File "<ipython-input-5-e8d018404dec>", line 6, in train
    for batch_idx, (data, target) in enumerate(train_loader):
  File "/home/imatge_upc_ai/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 272, in __next__
    return self._process_next_batch(batch)
  File "/home/imatge_upc_ai/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 305, in _process_next_batch
    self._put_indices()
  File "/home/imatge_upc_ai/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 298, in _put_indices
    self.index_queues[self.worker_queue_idx].put((self.send_idx, indices))
  File "/home/imatge_upc_ai/anaconda3/lib/python3.6/multiprocessing

KeyboardInterrupt: 